In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.1
sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)
matplotlib 3.1.3
numpy 1.18.1
pandas 1.0.1
sklearn 0.22.1
tensorflow 2.0.1
tensorflow_core.keras 2.2.4-tf


In [2]:
# demo has 4 gpus
tf.debugging.set_log_device_placement(True)
tf.config.set_soft_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print(len(gpus))   # 4
logical_gpus = tf.config.experimental.list_logical_devices('GPU')
print(len(logical_gpus))  # 4

[]
0
0


In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]
print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)


(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [3]:
# 归一化 x = (x - u) / std

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)
x_valid_scaled = scaler.transform(x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)
x_test_scaled = scaler.transform(x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)

In [5]:
def make_dataset(images, labels, epochs, batch_size, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size).prefetch(50)
    return dataset

batch_size = 128
epochs = 100
train_dataset = make_dataset(x_train_scaled, y_train, epochs, batch_size)

In [6]:
# tf.keras.models.Sequential()
model = keras.models.Sequential()
# filters: 卷积核数
# kernel_size: 卷积核的大小
# padding: same  补０使输入和输出的大小一样
# "selu"因为自带规一化，效果更好，可以更快地达到较好的训练效果，就不用自己进行数据规一化了
with tf.device(logical_gpus[0].name):
    model.add(keras.layers.Conv2D(filters=8, kernel_size=3,padding='same', activation='relu',input_shape=(28, 28, 1)))
    model.add(keras.layers.Conv2D(filters=8, kernel_size=3,padding='same', activation='relu'))
    model.add(keras.layers.MaxPool2D(pool_size=2))
    # 经过pool之后，数据缩小了pool_size倍，会造成数据损失，为了缓解数据损失，所以下一个卷积参数放大pool_size倍
    model.add(keras.layers.Conv2D(filters=16, kernel_size=3,padding='same', activation='relu'))
    model.add(keras.layers.Conv2D(filters=16, kernel_size=3,padding='same', activation='relu'))
    model.add(keras.layers.MaxPool2D(pool_size=2))
with tf.device(logical_gpus[1].name):
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3,padding='same', activation='relu'))
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3,padding='same', activation='relu'))
    model.add(keras.layers.MaxPool2D(pool_size=2))
    model.add(keras.layers.Flatten())
with tf.device(logical_gpus[2].name):
    model.add(keras.layers.Dense(32, activation="relu"))
    model.add(keras.layers.Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 8)         80        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 8)         584       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 16)        1168      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 16)        2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 16)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 32)          4

In [ ]:
history = model.fit(train_dataset,
                    steps_per_epoch = x_train_scaled.shape[0] // batch_size,
                    epochs=10)

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=[8, 5])
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()
    
plot_learning_curves(history)


In [ ]:
model.evaluate(x_test_scaled, y_test)